In [37]:
import folium
from shapely.geometry import LineString, MultiLineString, mapping
import json

# Helper function for handling Antimeridian crossing
def handle_antimeridian_crossing(points, tolerance=1e-6):
    if not points:
        return MultiLineString()

    linestrings = []
    current_linestring = [points[0]]

    for i in range(1, len(points)):
        prev_point, current_point = points[i - 1], points[i]
        if abs(prev_point[0] - current_point[0]) > 180 - tolerance:  # Check for antimeridian crossing
            # Split the line at the antimeridian
            if prev_point[0] < 0:
                split_point1 = (180, prev_point[1])
                split_point2 = (-180, current_point[1])
            else:
                split_point1 = (180, current_point[1])
                split_point2 = (-180, prev_point[1])

            # Add the first part of the split line to the list
            current_linestring.append(split_point1)
            linestrings.append(LineString(current_linestring))
            
            # Start a new line string with the second part of the split
            current_linestring = [split_point2, current_point]
        else:
            current_linestring.append(current_point)

    # Add the final linestring if it has points
    if len(current_linestring) > 1:
        linestrings.append(LineString(current_linestring))

    return MultiLineString(linestrings)

def create_multilinestring_route(points):
    if len(points) < 2:
        return MultiLineString()
    return handle_antimeridian_crossing(points)

# Example points for testing
points = [
    (159.19163, -26.19874),
    (172.24631, -5.5278),
    (-173.379596, -4.696802),
    (-152.744255, -6.856613),
    (-145.00739, -39.73823),
    (-150, -40),
    (-153, -7),
    (-175, -5),
    (170, -6),
    (160, -25),
    (-155, -41),
    (-154, -8),
    (-176, -6),
    (168, -7),
    (161, -24),
    (-160, -40),
    (-155, -9),
    (-177, -7),
    (167, -8),
    (162, -23)
]

route = create_multilinestring_route(points)

# Convert MultiLineString to GeoJSON format
def multiline_to_geojson(multiline):
    return mapping(multiline)

geojson = multiline_to_geojson(route)

# Save the GeoJSON to a file
with open('route.geojson', 'w') as f:
    json.dump(geojson, f)

print("GeoJSON saved as route.geojson")

# Create a Folium map centered at a point
m = folium.Map(location=[0, 0], zoom_start=2)

# Add the MultiLineString to the map
folium.GeoJson(
    geojson,
    name="Antimeridian Route",
    style_function=lambda x: {'color': 'blue', 'weight': 3}
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('route_map.html')

print("Map saved as route_map.html")


GeoJSON saved as route.geojson
Map saved as route_map.html
